In [1]:
import json
import torch
from functools import partial
from baukit import TraceDict
from einops import rearrange, einsum
from tqdm import tqdm

from cmap_utils import get_model_and_tokenizer, load_data, eval_model_performance, cmap_in, cmap_out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(10)

%load_ext autoreload
%autoreload 2

### Loading Models and Tokenizer

In [2]:
llama_model, tokenizer = get_model_and_tokenizer(model_name="llama", device=device)
goat_model, _ = get_model_and_tokenizer(model_name="goat", device=device)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

### Loading Data

In [3]:
data_file = "../data/dataset.jsonl"
dataloader = load_data(tokenizer=tokenizer, data_file=data_file, num_samples=500, batch_size=8)

Length of dataset: 500


### Models Performance

In [4]:
llama_acc = eval_model_performance(llama_model, dataloader, device)
goat_acc = eval_model_performance(goat_model, dataloader, device)

print(f"LLAMA accuracy: {llama_acc}")
print(f"Goat accuracy: {goat_acc}")

  0%|          | 0/63 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 63/63 [00:23<00:00,  2.63it/s]
63it [00:23,  2.63it/s]
  0%|          | 0/63 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 63/63 [00:24<00:00,  2.57it/s]
63it [00:24,  2.57it/s]

LLAMA accuracy: 0.66
Goat accuracy: 0.82


### Loading Model Activations

In [5]:
llama_modules = [[f"model.layers.{layer}.self_attn.k_proj", 
                  f"model.layers.{layer}.self_attn.q_proj",
                  f"model.layers.{layer}.self_attn.v_proj",
                 f"model.layers.{layer}.self_attn.o_proj"] 
                 for layer in range(llama_model.config.num_hidden_layers)]
goat_modules = [[f"base_model.model.model.layers.{layer}.self_attn.k_proj", 
                 f"base_model.model.model.layers.{layer}.self_attn.q_proj",
                 f"base_model.model.model.layers.{layer}.self_attn.v_proj",
                f"base_model.model.model.layers.{layer}.self_attn.o_proj"] 
                for layer in range(goat_model.config.num_hidden_layers)]

llama_modules = [item for sublist in llama_modules for item in sublist]
goat_modules = [item for sublist in goat_modules for item in sublist]

In [6]:
goat_cache = {}

with torch.no_grad():
    for bi, inputs in tqdm(enumerate(dataloader), desc="goat_cache"):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(goat_model.device)

        with TraceDict(goat_model, goat_modules, retain_input=True) as cache:
            _ = goat_model(inputs["input_ids"])
        
        for goat_layer, llama_layer in zip(goat_modules, llama_modules):
            if "o_proj" in llama_layer and "o_proj" in goat_layer:
                if bi in goat_cache:
                    goat_cache[bi][llama_layer] = cache[goat_layer].input.cpu()
                else:
                    goat_cache[bi] = {}
                    goat_cache[bi][llama_layer] = cache[goat_layer].input.cpu()
            else:
                if bi in goat_cache:
                    goat_cache[bi][llama_layer] = cache[goat_layer].output.cpu()
                else:
                    goat_cache[bi] = {}
                    goat_cache[bi][llama_layer] = cache[goat_layer].output.cpu()

goat_cache: 63it [00:46,  1.36it/s]


In [7]:
llama_cache = {}

with torch.no_grad():
    for bi, inputs in tqdm(enumerate(dataloader), desc="llama_cache"):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(llama_model.device)

        with TraceDict(llama_model, llama_modules, retain_input=True) as cache:
            _ = llama_model(inputs["input_ids"])
        
        for llama_layer in llama_modules:
            if "o_proj" in llama_layer:
                if bi in llama_cache:
                    llama_cache[bi][llama_layer] = cache[llama_layer].input.cpu()
                else:
                    llama_cache[bi] = {}
                    llama_cache[bi][llama_layer] = cache[llama_layer].input.cpu()
            else:
                if bi in llama_cache:
                    llama_cache[bi][llama_layer] = cache[llama_layer].output.cpu()
                else:
                    llama_cache[bi] = {}
                    llama_cache[bi][llama_layer] = cache[llama_layer].output.cpu()

llama_cache: 63it [00:41,  1.51it/s]


### Loading circuit

In [8]:
with open("../experiment_1/results/circuits/llama_circuit.json", "r") as f:
    circuit_heads = json.load(f)

In [9]:
with open("../experiment_2/results/DCM/llama_circuit/value_fetcher/object_value/0.01.txt", "r") as f:
    data = f.readlines()
    value_fetcher = json.loads(data[0].split(": ")[1])

with open("../experiment_2/results/DCM/llama_circuit/pos_transmitter/positional/0.01.txt", "r") as f:
    data = f.readlines()
    pos_transmitter = json.loads(data[0].split(": ")[1])

with open("../experiment_2/results/DCM/llama_circuit/pos_detector/positional/0.01.txt", "r") as f:
    data = f.readlines()
    pos_detector = json.loads(data[0].split(": ")[1])

struct_reader = circuit_heads["struct_reader"]

print(f"Value Fetcher Heads: {len(value_fetcher)}")
print(f"Heads affecting direct logit heads: {len(pos_transmitter)}")
print(f"Heads at query box token: {len(pos_detector)}")
print(f"Heads at prev query box token: {len(struct_reader)}")

Value Fetcher Heads: 40
Heads affecting direct logit heads: 5
Heads at query box token: 14
Heads at prev query box token: 5


### CMAP (output patching)

In [18]:
# Full circuit (Select group of heads for CMAP accordingly)
pos_heads_dict = {}
# pos_heads_dict[0] = value_fetcher
# pos_heads_dict[0] = pos_transmitter
# pos_heads_dict[2] = pos_detector
pos_heads_dict[-1] = struct_reader

In [19]:
correct_count, total_count = 0, 0

with torch.no_grad():
    for bi, inputs in tqdm(enumerate(dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(llama_model.device)

        with TraceDict(llama_model,
                       llama_modules,
                       retain_input=True,
                       edit_output=partial(
                            cmap_out,
                            model = llama_model,
                            goat_cache = goat_cache,
                            bi = bi,
                            pos_heads_dict = pos_heads_dict,
                            input_tokens = inputs)) as _:
                outputs = llama_model(inputs["input_ids"], output_attentions=True)

        for bi in range(inputs["labels"].size(0)):
            label = inputs["labels"][bi]
            pred = torch.argmax(outputs.logits[bi][inputs["last_token_indices"][bi]])

            if label == pred:
                correct_count += 1
            total_count += 1

        del outputs
        torch.cuda.empty_cache()

current_acc = round(correct_count / total_count, 2)
print(f"Task accuracy: {current_acc}")

63it [00:32,  1.93it/s]

Task accuracy: 0.67


Output CMAP Results:
- Full Circuit: 0.78
- Value Fetcher: 0.77
- Position Transmitter: 0.68
- Position Detector:0.62
- Structure Reader: 0.67

### CMAP (input patching)

In [28]:
# Select group of heads for CMAP accordingly
pos_heads_dict = {}
# pos_heads_dict[0] = value_fetcher
pos_heads_dict[0] = pos_transmitter

In [33]:
correct_count, total_count = 0, 0

with torch.no_grad():
    for bi, inputs in tqdm(enumerate(dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(llama_model.device)

        with TraceDict(llama_model,
                       llama_modules,
                       retain_input=True,
                       edit_output=partial(
                            cmap_in,
                            model = llama_model,
                            goat_cache = goat_cache,
                            llama_cache = llama_cache,
                            patching_component = ["q_proj", "k_proj", "v_proj"], #Options: "q_proj" (query), "k_proj" (key), "v_proj" (value)
                            bi = bi,
                            pos_heads_dict = pos_heads_dict,
                            input_tokens = inputs)) as _:
                outputs = llama_model(inputs["input_ids"], output_attentions=True)

        for bi in range(inputs["labels"].size(0)):
            label = inputs["labels"][bi]
            pred = torch.argmax(outputs.logits[bi][inputs["last_token_indices"][bi]])

            if label == pred:
                correct_count += 1
            total_count += 1

        del outputs
        torch.cuda.empty_cache()

current_acc = round(correct_count / total_count, 2)
print(f"Task accuracy: {current_acc}")

63it [00:38,  1.65it/s]

Task accuracy: 0.67


Input CMAP Results:
- Value Fetcher:
    - Query: 0.77
    - Key: 0.63
    - Value: 0.69
    - QK: 0.76
    - QKV: 0.77
- Position Transmitter:
    - Query: 0.66
    - Key: 0.65
    - Value: 0.69
    - QK: 0.65
    - QKV: 0.67

## Patching Weights

In [11]:
def patch_weights(inputs=None, output=None, layer=None, bi=None):
    inputs = inputs[0]
    value_fetcher_curr_layer = [h for l, h in value_fetcher if l == int(layer.split(".")[2])]
    position_trans_curr_layer = [h for l, h in pos_transmitter if l == int(layer.split(".")[2])]

    if ("q_proj" in layer) and len(value_fetcher_curr_layer) > 0:
        llama_w = llama_model.state_dict()[f"{layer}.weight"].clone()
        goat_w = goat_model.state_dict()[f"{layer}.weight"].clone()

        llama_w = rearrange(llama_w, 
                           "d_model (n_heads d_head) -> d_model n_heads d_head", 
                           n_heads=llama_model.config.num_attention_heads,
                           )
        goat_w = rearrange(goat_w, 
                           "d_model (n_heads d_head) -> d_model n_heads d_head", 
                           n_heads=goat_model.config.num_attention_heads,
                           )

#         for head_idx in value_fetcher_curr_layer:
        llama_w = goat_w

        llama_w = rearrange(llama_w, 
                           "d_model n_heads d_head -> d_model (n_heads d_head)", 
                           n_heads=llama_model.config.num_attention_heads,
                           )

        new_output = einsum(
            inputs, llama_w, "batch seq_len hidden_size, d_model hidden_size -> batch seq_len d_model"
        )
        output = torch.cat((output[:, :-1], new_output[:, -1].unsqueeze(dim=1)), dim=1)

    if ("o_proj" in layer) and len(position_trans_curr_layer) > 0:
        inputs = rearrange(
                inputs,
                "batch seq_len (n_heads d_head) -> batch seq_len n_heads d_head",
                n_heads=llama_model.config.num_attention_heads,
            )
        
        g_cache = rearrange(
                goat_cache[bi][layer],
                "batch seq_len (n_heads d_head) -> batch seq_len n_heads d_head",
                n_heads=llama_model.config.num_attention_heads,
            )
        
        for head in position_trans_curr_layer:
            inputs[:, -1, head] = g_cache[:, -1, head]
        
        inputs = rearrange(
                inputs,
                "batch seq_len n_heads d_head -> batch seq_len (n_heads d_head)",
                n_heads=llama_model.config.num_attention_heads,
            )
        w_o = llama_model.state_dict()[f"{layer}.weight"]
        output = einsum(
            inputs, w_o, "batch seq_len hidden_size, d_model hidden_size -> batch seq_len d_model"
        )

    return output

In [12]:
correct_count, total_count = 0, 0
llama_model.eval()
with torch.no_grad():
    for bi, inputs in tqdm(enumerate(tqdm(dataloader))):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(llama_model.device)

        with TraceDict(
            llama_model,
            llama_modules,
            retain_input=True,
            edit_output=partial(
                patch_weights,
                bi=bi,
            ),
        ) as _:
            outputs = llama_model(inputs["input_ids"])

        for bi in range(inputs["labels"].size(0)):
            label = inputs["labels"][bi]
            pred = torch.argmax(outputs.logits[bi][inputs["last_token_indices"][bi]])

            if label == pred:
                correct_count += 1
            # else:
            #     print(f"Label: {tokenizer.decode(label)}, Prediction: {tokenizer.decode(pred)}")
            total_count += 1

        del outputs
        torch.cuda.empty_cache()

  0%|          | 0/63 [00:00<?, ?it/s]
0it [00:00, ?it/s]

model.layers.14.self_attn.q_proj: llama: 108.36489868164062, fine-tuned: 108.38367462158203
model.layers.15.self_attn.q_proj: llama: 108.3883056640625, fine-tuned: 108.40525817871094
model.layers.16.self_attn.q_proj: llama: 106.8139419555664, fine-tuned: 106.8269271850586
model.layers.17.self_attn.q_proj: llama: 104.55338287353516, fine-tuned: 104.56761932373047
model.layers.18.self_attn.q_proj: llama: 103.43733215332031, fine-tuned: 103.45516967773438
model.layers.19.self_attn.q_proj: llama: 101.43477630615234, fine-tuned: 101.44762420654297
model.layers.20.self_attn.q_proj: llama: 102.77690887451172, fine-tuned: 102.79048156738281
model.layers.21.self_attn.q_proj: llama: 99.40530395507812, fine-tuned: 99.41356658935547
model.layers.22.self_attn.q_proj: llama: 101.00042724609375, fine-tuned: 101.01142883300781
model.layers.23.self_attn.q_proj: llama: 97.8895492553711, fine-tuned: 97.90031433105469
model.layers.24.self_attn.q_proj: llama: 97.97964477539062, fine-tuned: 98.0158920288086


  2%|▏         | 1/63 [00:00<00:26,  2.32it/s]

model.layers.14.self_attn.q_proj: llama: 108.36489868164062, fine-tuned: 108.38367462158203
model.layers.15.self_attn.q_proj: llama: 108.3883056640625, fine-tuned: 108.40525817871094
model.layers.16.self_attn.q_proj: llama: 106.8139419555664, fine-tuned: 106.8269271850586
model.layers.17.self_attn.q_proj: llama: 104.55338287353516, fine-tuned: 104.56761932373047
model.layers.18.self_attn.q_proj: llama: 103.43733215332031, fine-tuned: 103.45516967773438
model.layers.19.self_attn.q_proj: llama: 101.43477630615234, fine-tuned: 101.44762420654297
model.layers.20.self_attn.q_proj: llama: 102.77690887451172, fine-tuned: 102.79048156738281
model.layers.21.self_attn.q_proj: llama: 99.40530395507812, fine-tuned: 99.41356658935547
model.layers.22.self_attn.q_proj: llama: 101.00042724609375, fine-tuned: 101.01142883300781
model.layers.23.self_attn.q_proj: llama: 97.8895492553711, fine-tuned: 97.90031433105469
model.layers.24.self_attn.q_proj: llama: 97.97964477539062, fine-tuned: 98.0158920288086


  3%|▎         | 2/63 [00:00<00:25,  2.35it/s]

...

model.layers.14.self_attn.q_proj: llama: 108.36489868164062, fine-tuned: 108.38367462158203
model.layers.15.self_attn.q_proj: llama: 108.3883056640625, fine-tuned: 108.40525817871094
model.layers.16.self_attn.q_proj: llama: 106.8139419555664, fine-tuned: 106.8269271850586
model.layers.17.self_attn.q_proj: llama: 104.55338287353516, fine-tuned: 104.56761932373047
model.layers.18.self_attn.q_proj: llama: 103.43733215332031, fine-tuned: 103.45516967773438
model.layers.19.self_attn.q_proj: llama: 101.43477630615234, fine-tuned: 101.44762420654297
model.layers.20.self_attn.q_proj: llama: 102.77690887451172, fine-tuned: 102.79048156738281
model.layers.21.self_attn.q_proj: llama: 99.40530395507812, fine-tuned: 99.41356658935547
model.layers.22.self_attn.q_proj: llama: 101.00042724609375, fine-tuned: 101.01142883300781
model.layers.23.self_attn.q_proj: llama: 97.8895492553711, fine-tuned: 97.90031433105469
model.layers.24.self_attn.q_proj: llama: 97.97964477539062, fine-tuned: 98.0158920288086


 97%|█████████▋| 61/63 [00:25<00:00,  2.38it/s]

model.layers.14.self_attn.q_proj: llama: 108.36489868164062, fine-tuned: 108.38367462158203
model.layers.15.self_attn.q_proj: llama: 108.3883056640625, fine-tuned: 108.40525817871094
model.layers.16.self_attn.q_proj: llama: 106.8139419555664, fine-tuned: 106.8269271850586
model.layers.17.self_attn.q_proj: llama: 104.55338287353516, fine-tuned: 104.56761932373047
model.layers.18.self_attn.q_proj: llama: 103.43733215332031, fine-tuned: 103.45516967773438
model.layers.19.self_attn.q_proj: llama: 101.43477630615234, fine-tuned: 101.44762420654297
model.layers.20.self_attn.q_proj: llama: 102.77690887451172, fine-tuned: 102.79048156738281
model.layers.21.self_attn.q_proj: llama: 99.40530395507812, fine-tuned: 99.41356658935547
model.layers.22.self_attn.q_proj: llama: 101.00042724609375, fine-tuned: 101.01142883300781
model.layers.23.self_attn.q_proj: llama: 97.8895492553711, fine-tuned: 97.90031433105469
model.layers.24.self_attn.q_proj: llama: 97.97964477539062, fine-tuned: 98.0158920288086


100%|██████████| 63/63 [00:26<00:00,  2.38it/s]
63it [00:26,  2.38it/s]

model.layers.14.self_attn.q_proj: llama: 108.36489868164062, fine-tuned: 108.38367462158203
model.layers.15.self_attn.q_proj: llama: 108.3883056640625, fine-tuned: 108.40525817871094
model.layers.16.self_attn.q_proj: llama: 106.8139419555664, fine-tuned: 106.8269271850586
model.layers.17.self_attn.q_proj: llama: 104.55338287353516, fine-tuned: 104.56761932373047
model.layers.18.self_attn.q_proj: llama: 103.43733215332031, fine-tuned: 103.45516967773438
model.layers.19.self_attn.q_proj: llama: 101.43477630615234, fine-tuned: 101.44762420654297
model.layers.20.self_attn.q_proj: llama: 102.77690887451172, fine-tuned: 102.79048156738281
model.layers.21.self_attn.q_proj: llama: 99.40530395507812, fine-tuned: 99.41356658935547
model.layers.22.self_attn.q_proj: llama: 101.00042724609375, fine-tuned: 101.01142883300781
model.layers.23.self_attn.q_proj: llama: 97.8895492553711, fine-tuned: 97.90031433105469
model.layers.24.self_attn.q_proj: llama: 97.97964477539062, fine-tuned: 98.0158920288086

In [13]:
round(correct_count/total_count, 2)

0.66

## Patching inputs of Value Fetcher Heads

In [40]:
goat_input_cache = {}

with torch.no_grad():
    for bi, inputs in tqdm(enumerate(dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(goat_model.device)

        with TraceDict(goat_model, llama_modules, retain_input=True) as cache:
            _ = goat_model(inputs["input_ids"])

        for _, llama_layer in zip(goat_modules, llama_modules):
            if "o_proj" not in llama_layer:
                if bi in goat_input_cache:
                    goat_input_cache[bi][llama_layer] = cache[llama_layer].input.cpu()
                else:
                    goat_input_cache[bi] = {}
                    goat_input_cache[bi][llama_layer] = cache[llama_layer].input.cpu()

63it [00:36,  1.71it/s]


In [76]:
def patch_inputs(inputs=None, output=None, layer=None, bi=None):
    inp = inputs[0]
    value_fetcher_curr_layer = [h for l, h in value_fetcher if l == int(layer.split(".")[2])]
    position_trans_curr_layer = [h for l, h in pos_transmitter if l == int(layer.split(".")[2])]

    if ("k_proj" in layer) and len(value_fetcher_curr_layer) > 0:
        # Patching inputs and weights of value fetcher heads from fine-tuned to llama

        goat_inp = goat_input_cache[bi][layer]

        llama_w = llama_model.state_dict()[f"{layer}.weight"].clone()
        llama_w = rearrange(llama_w, 
                           "(n_heads d_head) d_model -> n_heads d_head d_model", 
                           n_heads=goat_model.config.num_attention_heads)
        
        goat_w = goat_model.state_dict()[f"{layer}.weight"].clone()
        goat_w = rearrange(goat_w, 
                           "(n_heads d_head) d_model -> n_heads d_head d_model", 
                           n_heads=goat_model.config.num_attention_heads)

        output = rearrange(output, 
                   "batch seq_len (n_heads d_head) -> batch seq_len n_heads d_head", 
                   n_heads=goat_model.config.num_attention_heads)

        for head_idx in value_fetcher_curr_layer:
            res = einsum(goat_inp.cuda(),
                          llama_w[head_idx, :],
                          "batch seq_len hidden_size, d_head hidden_size -> batch seq_len d_head")
            output[:, -1, head_idx] = res[:, -1]

        output = rearrange(output, 
                       "batch seq_len n_heads d_head -> batch seq_len (n_heads d_head)", 
                       n_heads=llama_model.config.num_attention_heads)

#     if ("o_proj" in layer) and len(position_trans_curr_layer) > 0:
#         inp = rearrange(
#                 inp,
#                 "batch seq_len (n_heads d_head) -> batch seq_len n_heads d_head",
#                 n_heads=llama_model.config.num_attention_heads)
        
#         g_cache = rearrange(
#                 goat_cache[bi][layer],
#                 "batch seq_len (n_heads d_head) -> batch seq_len n_heads d_head",
#                 n_heads=llama_model.config.num_attention_heads)
        
#         for head in position_trans_curr_layer:
#             inp[:, -1, head] = g_cache[:, -1, head]
        
#         inp = rearrange(
#                 inp,
#                 "batch seq_len n_heads d_head -> batch seq_len (n_heads d_head)",
#                 n_heads=llama_model.config.num_attention_heads)
#         w_o = llama_model.state_dict()[f"{layer}.weight"]
#         output = einsum(
#             inp, w_o, "batch seq_len hidden_size, d_model hidden_size -> batch seq_len d_model")

    return output

In [77]:
correct_count, total_count = 0, 0
with torch.no_grad():
    for bi, inputs in tqdm(enumerate(tqdm(dataloader))):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(llama_model.device)

        with TraceDict(
            llama_model,
            llama_modules,
            retain_input=True,
            edit_output=partial(
                patch_inputs,
                bi=bi,
            ),
        ) as _:
            outputs = llama_model(inputs["input_ids"])

        for bi in range(inputs["labels"].size(0)):
            label = inputs["labels"][bi]
            pred = torch.argmax(outputs.logits[bi][inputs["last_token_indices"][bi]])

            if label == pred:
                correct_count += 1
            # else:
            #     print(f"Label: {tokenizer.decode(label)}, Prediction: {tokenizer.decode(pred)}")
            total_count += 1

        del outputs
        torch.cuda.empty_cache()

  0%|          | 0/63 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 63/63 [00:27<00:00,  2.33it/s]
63it [00:27,  2.33it/s]


In [78]:
round(correct_count/total_count, 2)

0.65

In [85]:
for layer in llama_modules:
    llama_w = llama_model.state_dict()[f"{layer}.weight"].clone()
    goat_w = goat_model.state_dict()[f"{layer}.weight"].clone()
    value_fetcher_curr_layer = [h for l, h in value_fetcher if l == int(layer.split(".")[2])]

    if len(value_fetcher_curr_layer) > 0:
        print(f"Layer: {layer}, Llama weight: {llama_w.norm().item()}, Fine-tuned weight: {goat_w.norm().item()}")
        print(f"Layer: {layer}, Distance: {torch.dist(llama_w, goat_w)}\n")

Layer: model.layers.14.self_attn.k_proj, Llama weight: 109.2076416015625, Fine-tuned weight: 109.2258071899414
Layer: model.layers.14.self_attn.k_proj, Distance: 3.194016456604004

Layer: model.layers.14.self_attn.q_proj, Llama weight: 108.36489868164062, Fine-tuned weight: 108.38367462158203
Layer: model.layers.14.self_attn.q_proj, Distance: 3.2767512798309326

Layer: model.layers.14.self_attn.v_proj, Llama weight: 77.7557373046875, Fine-tuned weight: 77.76526641845703
Layer: model.layers.14.self_attn.v_proj, Distance: 2.292689085006714

Layer: model.layers.14.self_attn.o_proj, Llama weight: 77.85401916503906, Fine-tuned weight: 77.86835479736328
Layer: model.layers.14.self_attn.o_proj, Distance: 2.7456164360046387

Layer: model.layers.15.self_attn.k_proj, Llama weight: 110.20281982421875, Fine-tuned weight: 110.21894073486328
Layer: model.layers.15.self_attn.k_proj, Distance: 3.171581983566284

Layer: model.layers.15.self_attn.q_proj, Llama weight: 108.3883056640625, Fine-tuned weigh